In [67]:
import ee
import geemap
import pandas as pd

In [ ]:
def initialize_gee():
    """
    Authenticate and initialize the Google Earth Engine API.
    """
    try:
        ee.Initialize()
        print("Google Earth Engine initialized successfully.")
    except Exception as e:
        print("Authentication required. Proceeding to authenticate...")
        ee.Authenticate()
        ee.Initialize()
        print("Google Earth Engine authenticated and initialized successfully.")

In [37]:
# Call the function to initialize GEE
initialize_gee()

Google Earth Engine initialized successfully.


In [40]:

def display_mtbs_burn_severity(start_date, end_date, bbox):
    """
    Display the MTBS burn severity map within a specified date range and bounding box.

    Parameters:
    - start_date (str): The start date in 'YYYY-MM-DD' format.
    - end_date (str): The end date in 'YYYY-MM-DD' format.
    - bbox (list): Bounding box as [min_lon, min_lat, max_lon, max_lat].
    """

    # Load the MTBS burn severity dataset
    mtbs = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")

    # Filter the dataset by the specified date range
    mtbs_filtered = mtbs.filterDate(start_date, end_date)

    # Define the AOI using the provided bbox
    aoi = ee.Geometry.Rectangle(bbox)

    # Define visualization parameters
    vis_params = {
        'bands': ['Severity'],
        'min': 1,
        'max': 4,
        'palette': ['white', 'green', 'yellow', 'red']  # Colors for unburned, low, moderate, and high severity
    }

    # Create a map centered on the AOI
    center_lat = (bbox[1] + bbox[3]) / 2
    center_lon = (bbox[0] + bbox[2]) / 2
    map_ = geemap.Map(center=[center_lat, center_lon], zoom=6)

    # Add the filtered burn severity layer to the map
    map_.addLayer(mtbs_filtered.median().clip(aoi), vis_params, f"MTBS Burn Severity {start_date} to {end_date}")

    # Add the AOI boundary to the map
    map_.addLayer(aoi, {'color': 'blue'}, "AOI")

    # Define legend details based on the class table
    legend_dict = {
        'Background': '#000000',
        'Unburned to Low': '#006400',
        'Low': '#7fffd4',
        'Moderate': '#ffff00',
        'High': '#ff0000',
        'Increased Greenness': '#7fff00',
        'Non-Mapping Area': '#ffffff'
    }

    # Add the legend to the map
    map_.add_legend(title="MTBS Burn Severity Legend", legend_dict=legend_dict)

    # Display the map
    return map_


In [ ]:

# Example usage of the function
start_date = '2016-01-01'
end_date = '2023-12-31'
bbox = [-124.4, 32.5, -114.1, 42.0]  # Bounding box for California

# Call the function and display the map
display_mtbs_burn_severity(start_date, end_date, bbox)

Map(center=[37.25, -119.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [58]:
def display_mtbs_boundaries(start_date, end_date, bbox):
    """
    Display the MTBS burned area boundaries within a specified date range and bounding box.

    Parameters:
    - start_date (str): The start date in 'YYYY-MM-DD' format.
    - end_date (str): The end date in 'YYYY-MM-DD' format.
    - bbox (list): Bounding box as [min_lon, min_lat, max_lon, max_lat].
    """

    # Load the MTBS burned area boundaries dataset
    mtbs_boundaries = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

    # Define the AOI using the provided bbox
    aoi = ee.Geometry.Rectangle(bbox)

    # Filter the dataset by the specified date range and AOI
    mtbs_boundaries_filtered = (
        mtbs_boundaries
        .filterBounds(aoi)
        .filterDate(start_date, end_date)

    )

    # Visualization parameters
    vis_params = {
        'fillColor': '#ff8a50',  # Fill color
        'color': '#ff5722',      # Outline color
        'width': 1.0             # Outline width
    }

    # Create a map centered on the AOI
    center_lat = (bbox[1] + bbox[3]) / 2
    center_lon = (bbox[0] + bbox[2]) / 2
    map_ = geemap.Map(center=[center_lat, center_lon], zoom=6)

    # Add the filtered burned area boundaries layer to the map
    map_.addLayer(mtbs_boundaries_filtered, vis_params, f"MTBS Burned Area Boundaries {start_date} to {end_date}")

    # Add the AOI boundary to the map for reference
    map_.addLayer(aoi, {'color': 'blue'}, 'AOI Boundary')

    # Define legend details
    legend_dict = {
        'Burned Area': '#ff8a50',
        'Boundary Outline': '#ff5722'
    }

    # Add the legend to the map
    map_.add_legend(title="MTBS Burned Area Boundaries Legend", legend_dict=legend_dict)

    # Display the map
    return map_


In [59]:

# Example usage of the function
start_date = '2016-01-01'
end_date = '2021-12-31'
bbox = [-124.4, 32.5, -114.1, 42.0]  # Bounding box for California

# Call the function and display the map
display_mtbs_boundaries(start_date, end_date, bbox)

Map(center=[37.25, -119.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [62]:
def display_mtbs_boundaries(bbox, start_date,end_date):
    """
    Display the MTBS burned area boundaries within a specified bounding box and date range.

    Parameters:
    - bbox (list): Bounding box as [min_lon, min_lat, max_lon, max_lat].
    - start_date (str): The start date in 'YYYY-MM-DD' format (default is '2016-01-01').
    - end_date (str): The end date in 'YYYY-MM-DD' format (default is '2021-12-31').
    """

    # Load the MTBS burned area boundaries dataset
    mtbs_boundaries = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

    # Define the AOI using the provided bbox
    aoi = ee.Geometry.Rectangle(bbox)

    # Filter the dataset by AOI
    mtbs_boundaries_filtered = mtbs_boundaries.filterBounds(aoi)

    # Visualization parameters
    vis_params = {
        'fillColor': '#ff8a50',  # Fill color
        'color': '#ff5722',      # Outline color
        'width': 1.0             # Outline width
    }

    # Create a map centered on the AOI
    center_lat = (bbox[1] + bbox[3]) / 2
    center_lon = (bbox[0] + bbox[2]) / 2
    map_ = geemap.Map(center=[center_lat, center_lon], zoom=6)

    # Add the filtered burned area boundaries layer to the map
    map_.addLayer(mtbs_boundaries_filtered, vis_params, f"MTBS Burned Area Boundaries ({start_date} to {end_date})")

    # Add the AOI boundary to the map for reference
    map_.addLayer(aoi, {'color': 'blue'}, 'AOI Boundary')

    # Display the map
    return map_



In [64]:
# Example usage of the function
bbox = [-124.4, 32.5, -114.1, 42.0]  # Bounding box for California
start_date='2016-01-01'
end_date='2021-12-31'

# Call the function and display the map
display_mtbs_boundaries(bbox,start_date,end_date)

Map(center=[37.25, -119.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [65]:
def display_mtbs_by_event_id(event_id):
    """
    Display the MTBS burned area boundary for a specific Event ID.

    Parameters:
    - event_id (str): The Event ID to filter the dataset by.
    """
    # Initialize GEE
    initialize_gee()

    # Load the MTBS burned area boundaries dataset
    dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

    # Define the field and value to filter by
    field_name = 'Event_ID'  # Field name to filter by

    # Filter the dataset by the specified Event ID
    filtered_feature = dataset.filter(ee.Filter.eq(field_name, event_id))

    # Check if the filtered dataset is empty
    count = filtered_feature.size().getInfo()
    if count == 0:
        print(f"No feature found with {field_name}: {event_id}")
        return None
    else:
        print(f"Displaying feature with {field_name}: {event_id}")

    # Visualization parameters
    vis_params = {
        'fillColor': '#ff8a50',  # Fill color for the boundary
        'color': '#ff5722',      # Outline color
        'width': 2.0             # Outline width
    }

    # Create a map object
    map_ = geemap.Map()

    # Add the filtered feature to the map
    map_.addLayer(filtered_feature, vis_params, f"{field_name}: {event_id}")

    # Zoom to the feature if it exists
    map_.centerObject(filtered_feature, zoom=10)

    # Display the map
    return map_


In [66]:

# Example usage of the function
event_id = 'CA3983912034520210702'  # Replace with your desired Event ID

# Call the function and display the map
display_mtbs_by_event_id(event_id)

Google Earth Engine initialized successfully.
Displaying feature with Event_ID: CA3983912034520210702


Map(center=[39.93642291484538, -120.17855459031658], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
def get_mtbs_properties(event_id):
    """
    Retrieve the properties of an MTBS burned area boundary feature based on Event ID.

    Parameters:
    - event_id (str): The Event ID to filter the dataset by.

    Returns:
    - pd.DataFrame: A DataFrame containing the feature's properties.
    """
    # Load the MTBS burned area boundaries dataset
    dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

    # Filter the dataset by the specified Event ID
    filtered_feature = dataset.filter(ee.Filter.eq('Event_ID', event_id)).first()

    # Check if the feature exists
    if filtered_feature is None:
        print(f"No feature found with Event_ID: {event_id}")
        return None
    else:
        # Get the feature's properties
        properties = filtered_feature.getInfo()['properties']

        # Convert the properties dictionary to a DataFrame
        df = pd.DataFrame([properties])

        return df


In [70]:
# Example usage of the function
event_id = 'CA3983912034520210702'  # Replace with your desired Event ID

# Call the function and get the DataFrame
df = get_mtbs_properties(event_id)

df

Google Earth Engine initialized successfully.


,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,Comment,Event_ID,High_T,Ig_Date,IncGreen_T,Incid_Name,...,Map_ID,Map_Prog,Mod_T,NoData_T,Perim_ID,Post_ID,Pre_ID,dNBR_offst,dNBR_stdDv,irwinID
0,Initial,108368,39.957,-120.181,CA3987512036820210703 is a duplicate of this f...,CA3983912034520210702,650,1625209200000,-150,SUGAR,...,10024011,MTBS,450,-970,,804303220210805,804303220200701,44,32,33088F36-F802-4500-86A4-82DA6A62425F
